In [1]:
import numpy as np
import torch
import copy
from metabci.brainda.algorithms.deep_learning import EEGNet_Sparse
from metabci.brainda.algorithms.utils.model_selection import (
    set_random_seeds,
    generate_kfold_indices, match_kfold_indices)
from metabci.brainda.datasets import Wang2016, BETA
# 三个SSVEP Datasets：Nakanishi2015、Wang2016、BETA
from metabci.brainda.paradigms import SSVEP
from skorch.helper import predefined_split

In [2]:
#**************************************************
# Benchmark数据集读取处理
#**************************************************
Bench_dataset = Wang2016()
subject_list = list(range(1, 21))  # 被试编号从1到20
for s in subject_list:
    Bench_dataset.data_path(subject=s, path="E:\\MetaBCI-master\\mne_data")  # 依次为每个被试设置路径
events = Bench_dataset.events.keys()
freq_list = [str(Bench_dataset.get_freq(event)) for event in events]  # 获得所有刺激的频率
freq_map = {i: freq for i, freq in enumerate(freq_list)}  # 标签到频率的映射

Bench_subjects = list(range(1, 21))  # 使用S1–S20被试进行训练
Bench_paradigm = SSVEP(
    channels=['POZ', 'PZ', 'PO3', 'PO5', 'PO4', 'PO6', 'O1', 'OZ', 'O2'],  # 选择电极通道
    intervals=[(0.14, 1.14)],  # 分析时间窗最长-0.5-5.5s ,0.0是刺激开始
    events=freq_list,  # 选择所有刺激频率
    srate=250  # 采样率
)

# add 5-90Hz bandpass filter in raw hook
# 对原始EEG信号做5-90Hz带通滤波。
def raw_hook(raw, caches):
    raw.filter(5, 90, l_trans_bandwidth=2, h_trans_bandwidth=5, phase='zero-double')
    caches['raw_stage'] = caches.get('raw_stage', -1) + 1
    return raw, caches

Bench_paradigm.register_raw_hook(raw_hook)



# 获取微调数据
X_Bench, y_Bench, meta_Bench = Bench_paradigm.get_data(
    Bench_dataset,
    subjects=Bench_subjects,
    return_concat=True,
    n_jobs=None,
    verbose=False
)

--------ssssss, /upload/yijun/S1.mat.7z
--------ssssss, /upload/yijun/S2.mat.7z
--------ssssss, /upload/yijun/S3.mat.7z
--------ssssss, /upload/yijun/S4.mat.7z
--------ssssss, /upload/yijun/S5.mat.7z
--------ssssss, /upload/yijun/S6.mat.7z
--------ssssss, /upload/yijun/S7.mat.7z
--------ssssss, /upload/yijun/S8.mat.7z
--------ssssss, /upload/yijun/S9.mat.7z
--------ssssss, /upload/yijun/S10.mat.7z
--------ssssss, /upload/yijun/S11.mat.7z
--------ssssss, /upload/yijun/S12.mat.7z
--------ssssss, /upload/yijun/S13.mat.7z
--------ssssss, /upload/yijun/S14.mat.7z
--------ssssss, /upload/yijun/S15.mat.7z
--------ssssss, /upload/yijun/S16.mat.7z
--------ssssss, /upload/yijun/S17.mat.7z
--------ssssss, /upload/yijun/S18.mat.7z
--------ssssss, /upload/yijun/S19.mat.7z
--------ssssss, /upload/yijun/S20.mat.7z
--------ssssss, /upload/yijun/S1.mat.7z
--------ssssss, /upload/yijun/S2.mat.7z
--------ssssss, /upload/yijun/S3.mat.7z
--------ssssss, /upload/yijun/S4.mat.7z
--------ssssss, /upload/yijun

In [ ]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_freezing.py # Bench 1e-4学习率，S1~S20  F1=40,D=2,F2=80,Epsilon=20,shared_ratio=0.3,alpha=1.0
# 所有折准确率: [0.68125, 0.6325, 0.6075, 0.6575, 0.655, 0.64375]
# 平均准确率： 0.6462500000000001
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(42)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
F1 = 25
D = 2
F2 = 50
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.5
alpha = 0
epsilon = 30

Bench_save_path = "Bench_eegnet_Sparse_dropout_1s_0_0.5_改.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
    estimator.set_params(optimizer__lr=1e-3,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

cuda
(4800, 9, 250)
train_ind数量: 4000, validate_ind数量: 800, test_ind数量: 800
  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.0248        3.7725       0.0312        3.6887     +  0.0010  1.5809
      2       0.0295        3.7252       0.0475        3.6781     +  0.0010  1.3899
      3       0.0372        3.6942       0.0625        3.6620     +  0.0010  1.4074
      4       0.0393        3.6825       0.0788        3.6408     +  0.0010  1.3896
      5       0.0530        3.6531       0.0950        3.6145     +  0.0010  1.3964
      6       0.0650        3.6254       0.1075        3.5805     +  0.0010  1.4158
      7       0.0813        3.5910       0.1125        3.5361     +  0.0010  1.3876
      8       0.0978        3.5374       0.1450        3.4848     +  0.0010  1.3831
      9       0.1237        3.4864       0.1613        3.4222     +  0.0010  1.4172


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = [22,24,27,28,31,34,35] # 测试使用 
pre_save_path = "Bench_eegnet_Sparse_dropout_1s_0_0.5_改.pth"
# 加载预训练模型
F1 = 8
D = 2
F2 = 16
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
shared_ratio = 0.5
alpha = 0
epsilon = 30
estimator = EEGNet_Sparse(F1,D,F2,n_channels,n_samples,n_classes,time_kernel_length,shared_ratio,alpha,epsilon)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc

# 先把每个被试的准确率转成百分比并保留一位小数
acc_list = [round(acc * 100, 1) for acc in subject_accs.values()]
# 逐一打印单个被试
for subj, acc in zip(test_subjects, acc_list):
    print(f"subject {subj} 的准确率: {acc:.1f}%")

# 打印所有被试准确率（字典仍是原始小数）
print("所有被试的准确率：", subject_accs)

# 基于百分比后的数值再算平均
mean_acc = np.mean(acc_list)
print(f"所有被试的平均准确率: {mean_acc:.1f}%")

# 额外一行：只显示所有被试准确率（百分数），逗号隔开
print("所有被试准确率（无名称）:", ",".join([f"{acc:.1f}" for acc in acc_list]))

cuda
--------ssssss, /upload/yijun/S22.mat.7z
--------ssssss, /upload/yijun/S24.mat.7z
--------ssssss, /upload/yijun/S27.mat.7z
--------ssssss, /upload/yijun/S28.mat.7z
--------ssssss, /upload/yijun/S31.mat.7z
--------ssssss, /upload/yijun/S34.mat.7z
--------ssssss, /upload/yijun/S35.mat.7z
subject 22 的准确率: 37.1%
subject 24 的准确率: 38.3%
subject 27 的准确率: 46.2%
subject 28 的准确率: 32.5%
subject 31 的准确率: 43.8%
subject 34 的准确率: 35.4%
subject 35 的准确率: 45.0%
所有被试的准确率： {'22': 0.37083333333333335, '24': 0.38333333333333336, '27': 0.4625, '28': 0.325, '31': 0.4375, '34': 0.3541666666666667, '35': 0.45}
所有被试的平均准确率: 39.8%
所有被试准确率（无名称）: 37.1,38.3,46.2,32.5,43.8,35.4,45.0


In [ ]:
from metabci.brainda.algorithms.deep_learning import EEGNet

In [ ]:
#**************************************************
# Benchmark数据集  EEGNET网络
# !python3.8 Bench_eegnet.py # Bench数据集 S1~S20 学习率1e-4，F1=40,D=2,F2=80
# 所有折准确率: [0.5725, 0.6275, 0.58625, 0.57875, 0.57375, 0.59]
# 平均准确率： 0.5881249999999999                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
#**************************************************
# 设置device（如果GPU可用则使用GPU，否则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# 设置随机种子
set_random_seeds(38)
kfold = 6
print(X_Bench.shape)  # (2160, 9, 1250)
indices = generate_kfold_indices(meta_Bench, kfold=kfold)

# 预训练阶段
F1 = 25
D = 2
F2 = 50
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50

Bench_save_path = "Bench_eegnet_改.pth"

pre_accs = []
best_test_acc = -1
best_state_dict = None

for k in range(kfold):
    # 初始化模型
    estimator = EEGNet(F1, D, F2, n_channels, n_samples, n_classes, time_kernel_length)
    estimator.set_params(optimizer__lr=1e-4,device=device) # optimizer__lr=3e-3,
    train_ind, validate_ind, test_ind = match_kfold_indices(k, meta_Bench, indices)
    # 合并train_ind和validate_ind作为新的训练集索引
    new_train_ind = np.concatenate([train_ind, validate_ind])
    new_valid_ind = test_ind  # 用test_ind作为验证集
    new_test_ind = test_ind   # 用test_ind作为测试集

    print(f"train_ind数量: {len(new_train_ind)}, validate_ind数量: {len(new_valid_ind)}, test_ind数量: {len(new_test_ind)}")
    valid_ds = torch.utils.data.TensorDataset(
        torch.tensor(X_Bench[new_valid_ind], dtype=torch.float64),
        torch.tensor(y_Bench[new_valid_ind], dtype=torch.long)
    )
    estimator.set_params(train_split=predefined_split(valid_ds))
    estimator.fit(X_Bench[new_train_ind], y_Bench[new_train_ind])
    # 计算验证集准确率
    val_preds = estimator.predict(X_Bench[new_valid_ind])
    val_acc = np.mean(val_preds == y_Bench[new_valid_ind])
    #print("当前折验证集准确率：", val_acc)
    # 测试集评估
    p_labels = estimator.predict(X_Bench[new_test_ind])
    p_freqs = [freq_map[label] for label in p_labels]
    y_freqs = [freq_map[label] for label in y_Bench[new_test_ind]]
    print("预测频率：", p_freqs)
    print("真实频率：", y_freqs)
    pre_accs.append(np.mean(p_labels == y_Bench[new_test_ind]))
    print(f"第{k+1}折准确率: {pre_accs[-1]}")
    # 保存最优模型
    if pre_accs[-1] > best_test_acc:
        best_test_acc = pre_accs[-1]
        best_state_dict = copy.deepcopy(estimator.module.state_dict())
print(f"所有折准确率: {pre_accs}")
print("平均准确率：", np.mean(pre_accs))
# 保存测试集上最优的模型
if best_state_dict is not None:
    torch.save(best_state_dict, Bench_save_path)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_subjects = list(range(21, 36)) # 测试使用 
pre_save_path = "Bench_eegnet_改.pth"
# 加载预训练模型
F1 = 25
D = 2
F2 = 50
n_channels = X_Bench.shape[1] # 输入信号的通道数
n_samples = X_Bench.shape[2] # 输入信号的采样点数，时长*采样率
n_classes = 40
time_kernel_length = 50
estimator = EEGNet(F1, D, F2, n_channels, n_samples, n_classes, time_kernel_length)
estimator.set_params(device=device)
estimator.initialize()  
estimator.module.load_state_dict(torch.load(pre_save_path))
subject_accs = {}

# 逐一验证每个被试
for test_subj in test_subjects:
    # 获取该被试的数据
    X, y, meta = Bench_paradigm.get_data(
        Bench_dataset,
        subjects=[test_subj],
        return_concat=True,
        n_jobs=None,
        verbose=False
    )
    
    # 直接用预训练模型预测
    preds = estimator.predict(X)
    acc = np.mean(preds == y)
    subject_accs[str(test_subj)] = acc
    print(f"subject {test_subj} 的准确率: {acc}")

# 统一打印所有被试的准确率
print("所有被试的准确率：", subject_accs)
mean_acc = np.mean(list(subject_accs.values()))
print(f"所有被试的平均准确率: {mean_acc:.4f}")